In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA # Higher version of langchain -> from langchain_classic.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# step 1 Load the data
loader = TextLoader('my_docs.txt')
documents = loader.load()

# step 2 split - chunks
text_splitter = CharacterTextSplitter(chunk_size=200,chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# step 3 embedding object
embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

# step 4 stores to vectordb
vectorstore  = FAISS.from_documents(docs,embeddings)

# step 5 create retrievalobject
retriever_obj = vectorstore.as_retriever()

# step 6 create llm object
llm_obj = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API_KEY"))

# step 7 prompt - Strict prompt - Strict RAG
my_prompt = PromptTemplate(
    input_variables=["context","question"],
    template="""
    You are a strict retrieval QA assistant.
    Answer only using the information given in the context below.
    if the answer is not present in the context, reply exactly with:
    "I don't know, the document doesnot contain this information."
    Context:
    {context}
    Question:
    {question}
    Answer:
    """
)

# step 8: QAChain
rag_chain = RetrievalQA.from_chain_type(llm = llm_obj,retriever=retriever_obj,chain_type_kwargs={"prompt":my_prompt})